# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup - - - - -  - - - - - - - - - - - 3/13/2020 - Randy Dettmer
import matplotlib.pyplot as plt
import openweathermapy as owm
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress
import scipy.stats as st

import os
import csv

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities 3/13/2019
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
print(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Create settings dictionary with information we're interested in - set temperature to fahrenheit - 3/13/2019
url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'
#settings = {"units": "imperial", "appid": weather_api_key}

query_url = f"{url}appid={weather_api_key}&units={units}&q="

# Initialize data frame to store data from API calls
wdata_df =pd.DataFrame(columns = ["City", "Country", "Lat", "Lng",
                              "Date", "Max Temp", "Humidity",
                              "Cloudiness","Wind Speed"])
#wdata_df.head()

In [ ]:
#test 3 3 3 wrong wrong wrong wrong wrong wrong

count = 1 # counter to count the number of calls to API
err = 0 # error count to record how many cities the API responds back with no data.
sets = 0 # counter to count the number of call sets
index = 0 # index for the data frame WeatherPy data frame

#Get current time - do we need this????
t0 = time.time()

print(f'Beginning Data Retrieval \n ........................')

for city in cities:
    try:

        print(f'Processing Record {count} of Set {sets} | {city}' )

        response = requests.get(query_url + city).json()

# Initialize data frame to store data from API calls
        wdata_df.loc[index, "City"] = response[0]["name"]
        wdata_df.loc[index, "Cloudiness"] = response[0]["clouds"]["all"]
        wdata_df.loc[index, "Country"] = response[0]["sys"]["country"]
        wdata_df.loc[index, "Date"] = response[0]["dt"]
        wdata_df.loc[index, "Max Temp"] = response[0]["main"]["temp_max"]
        wdata_df.loc[index, "Wind Speed"] = response[0]["wind"]["speed"]
        wdata_df.loc[index, "Lat"] = response[0]["coord"]["lat"]
        wdata_df.loc[index, "Lng"] = response[0]["coord"]["lon"]
        wdata_df.loc[index, "Humidity"] = response[0]["main"]["humidity"]
        index = index +1
    except Exception as e:
        print(e)
        err= err+1
    count = count+1
    if count == 53: # loop to limit 50 API calls per minute. The OpenWeatherMap API limit is 60 calls/min
        t1=time.time()
        count = 1
        sets = sets+1
        time.sleep(60-(t1-t0))
        t0=time.time()
print("end of stats loop")
print(f'Could not get data for {err} cities')

In [ ]:
#test again - api.openweathermap.org/data/2.5/weather?q={city name}&appid={your api key}
settings = {"units": "imperial", "appid": weather_api_key}
#imperial api.openweathermap.org/data/2.5/find?q=London&units=imperial
# base url "find?q=" 'city name' "&units=imperial" "&appid=" api key

#set up lists to hold response info
#url = "http://api.openweathermap.org/data/2.5/weather?q="
#units = "imperial"
#towns = ['ushuaia', 'inta', 'butaritari', 'barentsburg', 'new norfolk'] #test list
#cities = ["Paris", "London", "Oslo", "Bejing"]
#df = pd.DataFrame(cities)
#df.head()

#query_url = f"{url}appid={weather_api_key}&units={units}&q="

#cloudiness = []
#country = []
#date = []
#humidity = []
#lat = []
#lng = []
#max_temp = []
#wind_speed = []

wresponse = []

#loop thru cities
#for town in towns:
#for city in cities:
for index, row in df.iterrows():
    #response = requests.get(query_url + town).json()
    #response = requests.get(url + city + "&units=imperial" + "&appid=" + weather_api_key).json()
    #print(city)
    wresponse.append(owm.get_current(row[0], **settings))
    

    #print(f'current weather for obj: {response}')
    
    
    #city.append(response['name'])
    #cloudiness.append(response['clouds']['all'])
    #country.append(response['sys']['country'])
    #date.append(response['dt'])
    #humidity.append(response['main']['humidity'])
    #lat.append(response['coord']['lat'])
    #lng.append(response['coord']['lon'])
    #max_temp.append(response['main']['temp_max'])
    #wind_speed.append(response['wind']['speed'])
    
#print(f"latitude info: {lat}")
#print(f"temperature info: {max_temp}")

In [ ]:
#test 3 3 3 - this works so don't delete this page until next cell is working - 3/13/2019 - needs tested -
url = "http://api.openweathermap.org/data/2.5/weather?"

api_cnt = 1 # counter to count the number of calls to API
#no_data = 0 # error count to record how many cities the API responds back with no data.
group = 0 # counter to count the number of call sets
index = 0 # index for the data frame WeatherPy data frame

#Get current time - do we need this????
time0 = time.time()

print(f'Beginning Data Retrieval')
print(f'........................')

for city in cities:
    try:
        #city1 = city.replace(" ", "%20") # remove space in city name so the URL retrieved is not broken????? - needed?

        print(f'Processing Record {api_cnt} of Set {group} | {city}')
        #print(f'{url}{city}&units=imperial&APPID={weather_api_key}')
        response = requests.get(city, **settings)
        
        results = response['info'].json()
        
        wdata_df.loc[index, "City"] = info[0]["name"]
        wdata_df.loc[index, "Country"] = info[0]["sys.country"]
        wdata_df.loc[index, "Lat"] = info[0]["coord.lat"]
        wdata_df.loc[index, "Lng"] = info[0]["coord.lon"]       
        wdata_df.loc[index, "Date"] = info[0]["dt"]
        wdata_df.loc[index, "Max Temp"] = info[0]["main.temp_max"]
        wdata_df.loc[index, "Humidity"] = info[0]["main.humidity"]
        wdata_df.loc[index, "Cloudiness"] = info[0]["clouds.all"]
        wdata_df.loc[index, "Wind Speed"] = info[0]["wind.speed"]
     
        index = index +1
    except Exception as err:
        print(err)
#       no_data = no_data+1
    api_cnt = api_cnt +1
    if api_cnt == 55: # loop to limit 55 API calls per minute. The OpenWeatherMap API limit is 60 calls/min
        time1=time.time()
        api_cnt = 1
        group = group+1
        time.sleep(60-(time1-time0))
        time0=time.time()
        
print("Data Retreval Complete")
#print(f'Could not get data for {no_data} cities')

In [ ]:
# test 3 3 3 - 3/13/2019
wdata_df.head()

In [ ]:
#display data types - 3/13/2019
wdata_df.info()

In [ ]:
#determine data type and convert where necessary to run regression - 3/13/2019
#converet non-null objects to int or float

wdata_df['Lat'] = wdata_df['Lat'].astype(float)
wdata_df['Lng'] = wdata_df['Lng'].astype(float)
wdata_df['Max Temp'] = wdata_df['Max Temp'].astype(float)
wdata_df['Humidity'] = wdata_df['Humidity'].astype(int)
wdata_df['Cloudiness'] = wdata_df['Cloudiness'].astype(int)
wdata_df['Wind Speed'] = wdata_df['Wind Speed'].astype(float)

#display data types to verify change
wdata_df.info()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#test 3 3 3 - 3/13/2019
#specify the file to write to file - - - -  - - - - - - - -

# save to csv file
wdata_df.to_csv("wdata.csv", encoding='utf-8', index=False)

# Display the city weather data frame
wdata_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#plot lat vs max temp - 3/13/2019
plt.scatter(wdata_df.iloc[:,2], wdata_df.iloc[:,5], marker = 'o', facecolors = 'yellow', edgecolors = 'black')
plt.xlabel("Latitude")
plt.ylabel("Maximum Temperature")
plt.title("Latitude versus Maximum Temperature")
plt.grid()

#### Latitude vs. Cloudiness Plot

In [ ]:
#plot lat vs cloudiness - 3/13/2019
plt.scatter(wdata_df.iloc[:,2], wdata_df.iloc[:,7], marker = 'o', facecolors = 'blue', edgecolors = 'black')
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Latitude versus Cloudiness")
plt.grid()

#### Latitude vs. Wind Speed Plot

In [ ]:
#plot lat vs cloudiness - 3/13/2019
plt.scatter(wdata_df.iloc[:,2], wdata_df.iloc[:,8], marker = 'o', facecolors = 'green', edgecolors = 'black')
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Latitude versus Wind Speed")
plt.grid()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames - 3/13/2019
# Note I included the equator (where lat = 0) as part of the southern hemisphere
northern = wdata_df.loc[(wdata_df["Lat"] > 0), :]
nwdata_df = pd.DataFrame(northern)
southern = wdata_df.loc[(wdata_df["Lat"] <= 0), :]
swdata_df = pd.DataFrame(southern)
swdata_df.tail()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#plot northern hemisphere lat vs max temp - - - - 3/13/2019
x_values = nwdata_df.iloc[:,2]
y_values = nwdata_df.iloc[:,5]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (0,0), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Maximum Temperature in Degrees Fahrenheit")
plt.title("Northern Hemisphere Latitude versus Maximum Temperature")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#plot southern hemisphere lat vs max temp - - - - 3/13/2019
x_values = swdata_df.iloc[:,2]
y_values = swdata_df.iloc[:,5]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (-30,55), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Maximum Temperature in Degrees Fahrenheit")
plt.title("Southern Hemisphere Latitude versus Maximum Temperature")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#plot northern hemisphere lat vs humidity - - - - 3/13/2019
x_values = nwdata_df.iloc[:,2]
y_values = nwdata_df.iloc[:,6]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (40,20), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("Northern Hemisphere Latitude versus Humidity")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#plot southern hemisphere lat vs humidity - - - - 3/13/2019
x_values = swdata_df.iloc[:,2]
y_values = swdata_df.iloc[:,6]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (-55,20), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("Southern Hemisphere Latitude versus Humidity")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#plot northern hemisphere lat vs cloudiness - - - - 3/13/2019
x_values = nwdata_df.iloc[:,2]
y_values = nwdata_df.iloc[:,7]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (45,30), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Northern Hemisphere Latitude versus Cloudiness")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#plot southern hemisphere lat vs humidity - - - - 3/13/2019
x_values = swdata_df.iloc[:,2]
y_values = swdata_df.iloc[:,7]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (-57,13), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Southern Hemisphere Latitude versus Cloudiness")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#plot northern hemisphere lat vs wind speed - - - - 3/13/2019
x_values = nwdata_df.iloc[:,2]
y_values = nwdata_df.iloc[:,8]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (0,40), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Northern Hemisphere Latitude versus Wind Speed")
print(f"The r-squared is: {r_value}")
plt.grid()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#plot southern hemisphere lat vs wind speed - - - - 3/13/2019
x_values = swdata_df.iloc[:,2]
y_values = swdata_df.iloc[:,8]
slope, intercept, r_value, p_value, std_err = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq, (-55,20), fontsize=15, color="red")

plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Southern Hemisphere Latitude versus Wind Speed")
print(f"The r-squared is: {r_value}")
plt.grid()